In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, set_seed
from dataset import XingDataset
import json

model = AutoModelForCausalLM.from_pretrained("/root/holo/x/models/ft/Qwen3-4B-ft-full-2048左截断/checkpoint-992").to('cuda:0')
tokenizer = AutoTokenizer.from_pretrained("../../models/Qwen3-4B")

In [ ]:
d = XingDataset('../data/val.json').data

In [ ]:
d[0]

In [ ]:
idx = 23
model_inputs = tokenizer(d[idx]['prompt'].split('<|im_start|>')[0]+'<|im_start|>', return_tensors="pt").to(model.device)
generated_ids = model.generate(**model_inputs, max_new_tokens=1024)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 
output = tokenizer.decode(output_ids, skip_special_tokens=True).strip()

In [ ]:
json.loads(output.replace("'", '"').replace('False', 'false').replace('True', 'true'))

In [ ]:
json.loads(d[idx]['prompt'].split('<|im_start|>')[1].split('<|im_end|>')[0].replace("'", '"').replace('False', 'false').replace('True', 'true'))

In [ ]:
import json
from tqdm import tqdm

zui_output = '../output/zui/qwen3-8b-ft-valb_val.json'

with open(zui_output, 'r') as f:
    data = json.load(f)

match_idx = []
unmatch_idx = []

for idx, item in tqdm(enumerate(data)):
    for idx2, label in enumerate(item['case_judgment_label']):
        person = label['person_name']
        predict_zui = item['zui'][person]
        answer_zui = ''
        for i in [1,2,3,5,6]:
            if label[f'predicted_sentence{i}']['value']:
                answer_zui = label[f'predicted_sentence{i}']['desc'].split('（')[0]
        if label['predicted_sentence4']['value']:
            answer_zui += f' 缓刑'
        if answer_zui == predict_zui:
            match_idx.append(f'{idx} {idx2}')
        else:
            unmatch_idx.append(f'{idx} {idx2}')

print(len(match_idx), len(unmatch_idx))


In [ ]:
data[0]